In [1]:
# import dependencies
# import sys
# !{sys.executable} -m pip install seaborn
import matplotlib as mplt
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import glacierml as gl
import numpy as np
import warnings
from tensorflow.python.util import deprecation
import os
import logging
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML
import geopy
from sklearn.cluster import KMeans
import glacierml as gl

# display(HTML("<style>.container { width:85% !important; }</style>"))
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.set_option('display.max_columns', None)

pd.set_option('mode.chained_assignment', None)

RS = range(0,25,1)

print('currently running tensorflow version: ' + tf.__version__)

2022-08-19 08:18:07.695958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 08:18:07.695979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


currently running tensorflow version: 2.9.1


# ML Analysis

In [4]:
# select dataset to analyze

selected_dataset = 'df7'

if selected_dataset == 'df1':
    df1 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'n'
    )
    module = 'sm1'
    res = 'sr1'
    dataset = df1
    dataset.name = selected_dataset
if selected_dataset == 'df2':
    df2 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'off'
    )
    module = 'sm2'
    res = 'sr2'
    dataset = df2
    dataset.name = selected_dataset
if selected_dataset == 'df3':
    df3 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 1
    )
    module = 'sm3'
    res = 'sr3'
    dataset = df3
    dataset.name = selected_dataset
if selected_dataset == 'df4':
    df4 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 5
    )
    module = 'sm4'
    res = 'sr4'
    dataset = df4
    dataset.name = selected_dataset
if selected_dataset == 'df5':
    df5 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'off'
    )
    df5 = df5.drop('Zmed', axis = 1)
    dataset = df5
    dataset.name = selected_dataset
    module = 'sm5'
    res = 'sr5'
if selected_dataset == 'df6':
    df6 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'r',
        region_selection = 1,
        area_scrubber = 'off'
    )
    module = 'sm6'
    res = 'sr6'
    reg = df6['region'].iloc[-1]
    df6 = df6.drop('region', axis=1)
    dataset = df6 
    dataset.name = str('df6_' + str(reg))
if selected_dataset == 'df7':
    df7 = gl.data_loader(
        root_dir = '/home/simonhans/data/prethicktor/',
    )
    module = 'sm7'
    res = 'sr7'
    dataset = df7 
    dataset.name = 'df7'
    
if selected_dataset == 'df8':
    df8 = gl.data_loader()
    df8 = df8.drop('Zmed', axis = 1)
    module = 'sm8'
    res = 'sr8'
    dataset = df8 
    dataset.name = 'df8'

In [5]:
# calculate model parameters 
total_inputs = (len(dataset) * (len(dataset.columns) -1))
inputs = len(dataset.columns) - 1
print('total inputs = ' + str(total_inputs))

non_trainable_parameters = (len(dataset.columns)) + ((len(dataset.columns) - 1))
print('non-trainable parameters = ' + str(non_trainable_parameters))

layer_1 = 60
layer_2 = 46

total_parameters = (
    layer_1 * (inputs + 1)  + 
    (layer_2 * (layer_1 + 1)) + 
    ((1 * (layer_2 + 1))) +
    non_trainable_parameters
)
print('trainable parameters = ' + str(total_parameters - non_trainable_parameters))

total inputs = 3888
non-trainable parameters = 19
trainable parameters = 3453


In [6]:
# load deviations table 
rootdir = 'zults/'
predictions = pd.DataFrame()
deviations = pd.DataFrame()

for file in tqdm(os.listdir(rootdir)):
    if 'predictions' in file:
        file_reader = pd.read_csv(rootdir + file)
        predictions = predictions.append(file_reader, ignore_index = True)
    if 'deviations' in file:
        file_reader = pd.read_csv(rootdir + file)
        deviations = pd.concat([deviations, file_reader], ignore_index = True)
        
deviations = deviations.drop('Unnamed: 0', axis = 1)
predictions = predictions.drop('Unnamed: 0', axis = 1)
deviations['total parameters'] = deviations['total parameters'].astype(int)
deviations['trained parameters'] = deviations['trained parameters'].astype(int)
deviations['total inputs'] = deviations['total inputs'].astype(int)
deviations = deviations[
    (deviations['df'].str.contains(selected_dataset)) 
#     &
#     (deviations['layer architecture'] == '10-5')
#     &
#     (deviations['learning rate'] == 0.100)
#     &
#     (deviations['epochs'] == 50)
]
deviations['test - train'] = (
    abs(deviations['test mae avg'] - deviations['train mae avg'])
)
deviations = deviations.sort_values(
    [
#         'layer architecture',
#         'test - train',
#         'epochs',
        'test mae avg', 
        
#         'test predicted thickness std dev'
#         'layer architecture',
#         'learning rate',
#         'df',
#         'layer architecture'
    ]
)
deviations

100%|███████████████████████████████████████| 412/412 [00:00<00:00, 1439.82it/s]


,layer architecture,total parameters,trained parameters,total inputs,df,dropout,learning rate,validation split,epochs,test mae avg,train mae avg,test mae std dev,train mae std dev,test predicted thickness std dev,train predicted thickness std dev,test - train
132,64-48,3828,3809,3834,df7,1,0.100,0.2,100,19.618951,18.981482,3.454822,1.395948,6.849141,4.450141,0.637469
129,50-28,1976,1957,3834,df7,1,0.100,0.2,100,20.028465,19.527148,3.338884,1.291159,7.229381,5.083309,0.501317
113,64-48,3828,3809,3834,df7,0,0.100,0.2,100,20.059461,19.628291,2.549977,1.251391,8.163460,5.486979,0.431170
110,50-28,1976,1957,3834,df7,0,0.100,0.2,100,20.696209,20.214970,2.694641,1.677756,8.642095,5.592713,0.481239
130,50-28,1976,1957,3834,df7,1,0.010,0.2,100,22.561178,21.071632,3.331436,0.922440,4.072864,2.156377,1.489545
114,64-48,3828,3809,3834,df7,0,0.010,0.2,100,22.611158,20.974647,3.235544,0.937583,4.880827,2.390968,1.636512
133,64-48,3828,3809,3834,df7,1,0.010,0.2,100,22.648749,20.791936,3.469679,0.908162,4.394150,2.198093,1.856813
111,50-28,1976,1957,3834,df7,0,0.010,0.2,100,22.925221,21.113651,3.606206,1.024825,3.937835,2.152602,1.811569
116,50-28,1976,1957,3834,df7,0,0.100,0.2,50,23.132563,22.146909,3.610950,1.574109,7.137610,5.599342,0.985654
127,10-5,180,161,3834,df7,1,0.100,0.2,100,23.236795,21.930665,3.544364,1.420382,6.380354,5.019096,1.306131


In [ ]:
# load selected model and look at predicted accuracies of each random state
print('Please select index from deviations table to inspect further')

selection = int(input())

arch = deviations['layer architecture'].loc[selection]
top_learning_rate = deviations['learning rate'].loc[selection]
epochs = deviations['epochs'].loc[selection]
dropout = deviations['dropout'].loc[selection]
print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

dnn_model = {}
rootdir = 'saved_models/' + module + '/sm_' + arch + '/'


train_features, test_features, train_labels, test_labels = gl.data_splitter(dataset)
features = pd.concat([train_features, test_features], ignore_index = True)
labels = pd.concat([train_labels, test_labels], ignore_index = True)

fig = plt.figure(figsize=(25, 25))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(
    'Volume estimates using layer architecture ' + arch +
    ', learning rate: ' + str(top_learning_rate) + ', epochs: ' + str(epochs)
             , fontsize=18, y=1
            )
fig.patch.set_facecolor('w')

for n, rs in tqdm(enumerate(RS)):
    ax = plt.subplot(5, 5, n + 1)
    model_name = (
        str(arch) + 
        '_' + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )   
    
    model_path = (
        rootdir + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    dnn_model[model_name] = tf.keras.models.load_model(model_path)
    
    
    y = dnn_model[model_name].predict(features, verbose = 0)
    plt.plot(labels,y,'.')
    plt.plot((0,400),(0,400),'-')
    plt.xlabel('True Thickness (m)')
    plt.ylabel('Model Thickness (m)')
    ax.set_title('Random State ' +str(rs))
    plt.xlim((0,400))
    plt.ylim((0,400))
# plt.tight_layout()

fig_dir = (
    'figs/' + 
    module + 
    '/'   
) 
isdir = os.path.isdir(fig_dir)
if isdir == False:
    os.makedirs(fig_dir)
file_name_1 = (
    fig_dir +
    'predictions_' +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) +
    '.eps'
)


file_name_2 = (
    fig_dir +
    'predictions_' +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) +
    '.png'
)

plt.tight_layout()
# fig.savefig(file_name_1)
# fig.savefig(file_name_2)

In [ ]:
# load selected model loss curves

fig_dir = (
    'figs/' + 
    module + 
    '/'   
) 
isdir = os.path.isdir(fig_dir)
if isdir == False:
    os.makedirs(fig_dir)

print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

print(fig_dir)
rootdir = 'saved_results/' + res + '/sr_' + arch + '/'
print(rootdir)
dnn_history = {}
fig,ax=plt.subplots(1,1,figsize=(15,15))
fig.patch.set_facecolor('w')
fig.suptitle('Loss curves for layer architecture ' + 
             arch + ', learning rate: ' + str(top_learning_rate) + ', epochs: ' + str(epochs) 
             )
for n, rs in enumerate(RS): 
    ax = plt.subplot(5,5,n+1)
    history_name = (
        arch + 
        '_' +
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    model_name = (
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    dnn_history[model_name] = pd.read_csv(rootdir + model_name)

    ax.set_title('Random State: ' + str(rs))
    gl.plot_loss(dnn_history[model_name])
    
file_name_1 = (
    fig_dir +
    'loss_' +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) +
    '.eps'
)

file_name_2 = (
    fig_dir +
    'loss_' +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) +
    '.png'
)

plt.tight_layout()
# fig.savefig(file_name_1)
# fig.savefig(file_name_2)


In [2]:
df = gl.predictions_loader()

df = df[
    (df['dataframe'].str.contains('df7_')) 
    &
    (df['architecture'] == '10-5')
    &
    (df['learning rate'] == '0.010')
    &
    (df['epochs'] == '100')
]
x
df = gl.glathida_stats_adder(
    df,
    pth_1 = '/home/simonhans/data/prethicktor/regional_data/raw/',
    pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/',
    pth_3 = '/home/simonhans/data/prethicktor/regional_data/training_data/',
)
df

In [8]:
# compute residuals
residuals = df[[
    'dataframe',
    'vol_ratio',
    'vol_from_zero',
    'voldiff',
]]

residuals['percent error'] = abs(
   ( 1 - residuals['vol_ratio']) * 100
)
residuals['region'] = residuals['dataframe'].str[4:]
residuals = residuals.drop('dataframe', axis = 1)

In [10]:
# df = df.drop(18)
df

,vol,Area_RGI_iqr,Area_RGI_mean,Area_RGI_median,Area_RGI_std,Aspect_RGI_iqr,Aspect_RGI_mean,Aspect_RGI_median,Aspect_RGI_std,Lmax_RGI_iqr,Lmax_RGI_mean,Lmax_RGI_median,Lmax_RGI_std,Slope_RGI_iqr,Slope_RGI_mean,Slope_RGI_median,Slope_RGI_std,Zmax_RGI_iqr,Zmax_RGI_mean,Zmax_RGI_median,Zmax_RGI_std,Zmin_RGI_iqr,Zmin_RGI_mean,Zmin_RGI_median,Zmin_RGI_std,architecture,area,dataframe,epochs,h mean f,learning rate,region,tolerance,total variance,volf,mean thickness,voldiff,Area_GlaThiDa_mean,Aspect_GlaThiDa_mean,Lmax_GlaThiDa_mean,Slope_GlaThiDa_mean,Zmin_GlaThiDa_mean,Zmax_GlaThiDa_mean,Area_GlaThiDa_median,Aspect_GlaThiDa_median,Lmax_GlaThiDa_median,Slope_GlaThiDa_median,Zmin_GlaThiDa_median,Zmax_GlaThiDa_median,Area_GlaThiDa_std,Aspect_GlaThiDa_std,Lmax_GlaThiDa_std,Slope_GlaThiDa_std,Zmin_GlaThiDa_std,Zmax_GlaThiDa_std,ratio trainable,vol_ratio,vol_from_zero
147,5.989575,0.92400,4.259054,0.4430,50.491526,244.0,195.227967,226.0,122.682674,1259.00,1977.983111,950.0,4645.133781,9.000,22.642861,22.00,6.333523,563.00,1834.176898,1827.0,491.597716,605.00,1290.382731,1314.0,457.285531,10-5,73890.321,df7_01,100,218.0,0.010,01,4.92,2.438608,18.98,81.060341,-12.990425,31.746125,178.125000,7208.750000,18.625000,1505.125000,2278.000000,1.8260,206.0,2351.0,21.00,1579.5,2554.0,76.730266,116.309326,13254.758068,5.452924,735.136116,800.062341,0.000295,0.315573,0.684427
218,1.266035,0.47700,0.783060,0.1870,4.609983,261.0,154.641819,105.0,127.848895,701.00,920.344790,578.0,1225.462142,9.500,24.439044,24.20,7.380997,659.75,2403.221295,2340.0,564.812650,637.00,2056.899002,1965.0,605.903653,10-5,14444.330,df7_02,100,72.0,0.010,02,0.27,1.029544,1.06,87.649301,0.206035,2.062406,168.531250,2526.000000,21.178125,2241.687500,3116.937500,1.1850,162.5,2138.0,20.80,2007.5,3011.0,3.074268,120.333387,2238.034517,5.621317,601.644384,608.579182,0.001697,1.194373,0.194373
184,53.623194,6.79225,23.105548,1.7635,124.489624,240.0,181.533861,183.0,120.888736,3494.50,4468.925462,1886.5,7969.118795,7.700,15.441953,14.40,6.112923,486.25,1053.947669,1059.0,367.675504,417.25,510.087071,525.0,292.411531,10-5,105084.033,df7_03,100,270.0,0.010,03,7.35,3.898395,28.33,510.288690,25.293194,1238.533333,259.000000,54630.333333,6.100000,16.000000,1419.666667,999.8610,323.0,71606.0,6.30,0.0,1921.0,1046.519763,92.639085,32082.883332,0.509902,22.627417,819.563434,0.000658,1.892806,0.892806
49,11.730338,2.14750,5.514259,0.6570,43.879279,236.0,172.181254,163.0,120.474964,1924.50,2331.581389,1191.0,3683.474811,7.100,17.700000,16.90,5.800160,447.50,1119.713554,1120.0,315.732986,389.00,700.133378,695.0,303.851676,10-5,40888.228,df7_04,100,210.0,0.010,04,2.23,1.819072,8.61,286.887909,3.120338,1479.180000,30.500000,31318.000000,7.300000,420.500000,1452.000000,1479.1800,30.5,31318.0,7.30,420.5,1452.0,1292.232000,20.500000,1445.000000,3.700000,17.500000,335.000000,0.000270,1.362409,0.362409
133,13.086196,1.70700,6.420456,0.3930,79.442858,216.0,179.737868,177.0,114.502877,1606.00,2041.995063,838.0,3956.197049,12.900,20.430547,19.60,9.218062,668.00,1253.692304,1221.0,488.383456,610.00,781.527324,761.0,436.150937,10-5,130059.170,df7_05,100,175.0,0.010,05,4.07,5.206576,15.69,100.617251,-2.603804,20.154000,261.000000,8083.000000,11.300000,159.000000,983.000000,20.1540,261.0,8083.0,11.30,159.0,983.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000049,0.834047,0.165953
273,6.066283,2.57225,19.531337,0.5635,117.210684,229.0,170.358657,152.0,117.748556,2074.25,3148.473498,983.0,6831.996506,8.600,17.630212,16.75,6.573607,243.75,1294.558304,1267.0,247.310301,266.25,892.000000,935.5,252.111431,10-5,11054.737,df7_06,100,341.0,0.010,06,0.98,1.295514,3.77,548.749644,2.296283,608.658000,142.000000,28302.666667,11.366667,477.666667,1511.000000,264.5360,117.0,22676.0,7.30,607.0,1762.0,682.150351,41.880783,25153.662349,7.512360,262.994719,499.241425,0.005282,1.609094,0.609094
191,9.025042,7.09350,21.027204,1.8740,78.127422,234.0,174.302167,164.0,120.216076,3937.50,4961.415480,2471.0,6949.757330,6.700

In [89]:
residuals.sort_values('percent error')

,vol_ratio,vol_from_zero,voldiff,percent error,region
8,0.939719,0.060281,-0.882511,6.028081,09
4,0.834047,0.165953,-2.603804,16.595308,05
1,1.194373,0.194373,0.206035,19.437304,02
10,1.198431,0.198431,0.025796,19.843113,11
6,1.208172,0.208172,1.555042,20.817157,07
12,1.252420,0.252420,0.825415,25.242042,13
7,0.725387,0.274613,-0.082384,27.461288,08
3,1.362409,0.362409,3.120338,36.240862,04
16,0.616402,0.383598,-2.048412,38.359777,17
14,1.395338,0.395338,0.347898,39.533847,15


In [ ]:
# sns density plot
sns.set(rc={"figure.figsize":(15,10)})
sns.kdeplot(x = test_labels, y = y.flatten(),fill = True)
plt.plot((0,300),(0,300),'-')

In [ ]:
# not enough data to verify physical models from just data perspective